In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [52]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, StructField
from pyspark.sql import functions as F
from pyspark.sql import types as t
from pyspark.sql.functions import udf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import os

In [3]:
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars org.apache.spark:spark-sql-kafka-0-10_2.12:2.0.0 --packages org.apache.spark:spark-sql-kafka-0-10_2.12:2.0.0 pyspark-shell'
# Создаем объект SparkSession, который является точкой входа в Spark SQL и Spark Streaming API.

spark = SparkSession\
    .builder\
    .appName("MySparkApp")\
    .config("spark.jars.packages", 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.3')\
    .getOrCreate()

In [74]:
options = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="mlops" password="mlops_pw";',
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol" : "PLAINTEXT",
    "kafka.bootstrap.servers": '51.250.110.24:9092',
    "group.id": 'test_group',
    "subscribe": 'test',
}

df = spark \
  .readStream \
  .format("kafka") \
  .options(**options) \
  .load()

In [75]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [76]:
schema = t.StructType(
    [
        t.StructField('X', t.ArrayType(t.DoubleType()), True),
        t.StructField('y', t.FloatType(), True),
    ],
)

In [77]:
df = (df.selectExpr('CAST(value AS STRING)') \
      .select(from_json('value', schema).alias('raw_data')))
print("df schema")
df.printSchema()

df schema
root
 |-- raw_data: struct (nullable = true)
 |    |-- X: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- y: float (nullable = true)



In [78]:
df = df.withColumn('X_train', df.raw_data.X).withColumn('y_train', df.raw_data.y)

In [79]:
df.printSchema()

root
 |-- raw_data: struct (nullable = true)
 |    |-- X: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- y: float (nullable = true)
 |-- X_train: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- y_train: float (nullable = true)



In [80]:
df = df.withColumn('exploded', F.explode('X_train'))

In [81]:
vector_assembler = VectorAssembler(inputCols=["exploded"], outputCol="features")
vectorized_data = vector_assembler.transform(df)

In [83]:
vectorized_data

DataFrame[raw_data: struct<X:array<double>,y:float>, X_train: array<double>, y_train: float, exploded: double, features: vector]

In [82]:
# Создание и обучение модели линейной регрессии
linear_regression = LinearRegression(featuresCol="features", labelCol="y_train")
model = linear_regression.fit(vectorized_data)

# Предсказание на потоковых данных с помощью модели
predictions = model.transform(vectorized_data)

# Определение настроек для записи данных в Kafka

write_kafka_params = {
    # "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="mlops" password="mlops_pw";',
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol" : "PLAINTEXT",
    "kafka.bootstrap.servers": '51.250.110.24:9092',
    "topic": "test-topic"
}

# Запись предсказаний в Kafka
stream_writer = predictions.writeStream \
    .format("kafka") \
    .outputMode("append") \
    .options(**write_kafka_params) \
    .start()

# Запуск потока обработки
stream_writer.awaitTermination()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka

In [8]:
console = (console
           .writeStream
           .format('console')
           .queryName('console-output'))

In [ ]:
console.start().awaitTermination()

In [12]:
console.show(1)

AttributeError: 'DataStreamWriter' object has no attribute 'show'